In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

/home/zaza/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
PATH = 'data/grocery-sales/'

In [3]:
!ls {PATH}

holidays_events.csv.7z	sample_submission.csv.7z  train.csv
items.csv.7z		stores.csv.7z		  train.csv.7z
oil.csv.7z		test.csv.7z		  transactions.csv.7z


In [4]:
types = {'id': 'int64', 
        'itemnbr': 'int32',
        'store_nbr': 'int8',
        'unit_sales': 'float32',
        'onpromotion': 'object'}

In [ ]:
%time df_raw = pd.read_csv(f'{PATH}/train.csv', parse_dates=["date"], dtype = types, infer_datetime_format=True, nrows=100000000)

In [13]:
%time df_raw = pd.read_csv(f'{PATH}/train.csv', parse_dates=["date"], nrows=10000000)

CPU times: user 3.26 s, sys: 629 ms, total: 3.89 s
Wall time: 3.92 s


In [4]:
add_datepart(df_raw, 'date')

In [5]:
#train_cats(df_raw)

In [6]:
df_raw.unit_sales = np.log(df_raw.unit_sales)

/home/zaza/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
df_raw.head()

,id,store_nbr,item_nbr,unit_sales,onpromotion,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0,25,103665,1.945910,NaN,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
1,1,25,105574,0.000000,NaN,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
2,2,25,105575,0.693147,NaN,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
3,3,25,108079,0.000000,NaN,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
4,4,25,108701,0.000000,NaN,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400


In [8]:
df_raw.drop('onpromotion', axis=1, inplace=True)

In [11]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/grocery-raw')

In [12]:
df_raw = pd.read_feather('tmp/grocery-raw')

In [13]:
df, y, nas = proc_df(df_raw, 'unit_sales')

In [35]:
df.isna().sum().sort_index() #check how many null input in different columns, sorted


Day                 0
Dayofweek           0
Dayofyear           0
Elapsed             0
Is_month_end        0
Is_month_start      0
Is_quarter_end      0
Is_quarter_start    0
Is_year_end         0
Is_year_start       0
Month               0
Week                0
Year                0
id                  0
item_nbr            0
store_nbr           0
dtype: int64

In [28]:
m = RandomForestRegressor(n_jobs=-1)
#m.fit(df,y)
#m.score(df,y) #will get an overfitted predicition, need to split to train and validation set


In [36]:
m.fit(df,y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [103]:
df_raw.head()

,id,store_nbr,item_nbr,unit_sales,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0,25,103665,1.945910,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
1,1,25,105574,0.000000,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
2,2,25,105575,0.693147,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
3,3,25,108079,0.000000,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
4,4,25,108701,0.000000,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
